In [39]:
import sys
from datetime import time
import time
import ctypes
from enum import Enum
import win32com.client
import pandas as pd
from pykrx import stock

# Plus 공통 Object

In [40]:
g_objCodeMgr = win32com.client.Dispatch('CpUtil.CpCodeMgr')
g_objCpStatus = win32com.client.Dispatch('CpUtil.CpCybos')
g_objCpTrade = win32com.client.Dispatch('CpTrade.CpTdUtil')
g_objFutureMgr = win32com.client.Dispatch("CpUtil.CpFutureCode")
g_objKsdFMgr = win32com.client.Dispatch("CpUtil.CpKFutureCode")
g_objElwMgr = win32com.client.Dispatch("CpUtil.CpElwCode")
g_objOptionMgr = win32com.client.Dispatch("CpUtil.CpOptionCode")
g_objUsMgr = win32com.client.Dispatch("CpUtil.CpUsCode")


In [41]:
# PLUS 실행 기본 체크 함수
def InitPlusCheck():
    # 프로세스가 관리자 권한으로 실행 여부
    if ctypes.windll.shell32.IsUserAnAdmin():
        print('정상: 관리자권한으로 실행된 프로세스입니다.')
    else:
        print('오류: 일반권한으로 실행됨. 관리자 권한으로 실행해 주세요')
        return False

    # 연결 여부 체크
    if (g_objCpStatus.IsConnect == 0):
        print("PLUS가 정상적으로 연결되지 않음. ")
        return False

    # # 주문 관련 초기화 - 계좌 관련 코드가 있을 때만 사용
    # if (g_objCpTrade.TradeInit(0) != 0):
    #     pr"주문 초기화 실패")
    #     return False

    return True

In [42]:
InitPlusCheck()

정상: 관리자권한으로 실행된 프로세스입니다.


True

In [43]:

class Rqtype(Enum):
    ORDER = 0
    SISE = 1


def waitRqLimit(rqtype):

    remainCount = g_objCpStatus.GetLimitRemainCount(rqtype.value)

    if remainCount > 0:
        print('남은 횟수: ', remainCount)
        return True

    remainTime = g_objCpStatus.LimitRequestRemainTime
    print('조회 제한 회피 time wait %.2f초 ' % (remainTime / 1000.0))
    time.sleep(remainTime / 1000)
    return True



In [44]:

# ### CpSvr7254  투자주체별현황을 일별/기간별, 순매수/매매비중을 일자별로 확인할수 있습니다
def subCpSvr7254(m_code, m_FromDate, m_ToDate):
   # ## 대신 API 세팅
    cpSvr7254 = win32com.client.Dispatch("CpSysDib.CpSvr7254")
    cpSvr7254.SetInputValue(0, m_code)       # 종목코드
    cpSvr7254.SetInputValue(1, '0')          # 기간선택 0:기간선택, 1:1개월, ... , 4:6개월
    cpSvr7254.SetInputValue(2, m_FromDate)  # 시작일자
    cpSvr7254.SetInputValue(3, m_ToDate)    # 끝일자
    cpSvr7254.SetInputValue(4, '0')         # 0:순매수 1:비중
    cpSvr7254.SetInputValue(5, '0')         # 투자자
    # cpSvr7254.BlockRequest()

    # numData=cpSvr7254.GetHeaderValue(1)
    # print(numData)
    data=[]
    # for ixRow in range(numData):
    #     tempData=[]
    #     for ixCol in range(14):
    #         tempData.append(cpSvr7254.GetDataValue(ixCol, ixRow))
    #     data.append(tempData)

    # 연속 수행
    while True:
        waitRqLimit(Rqtype.SISE)
        cpSvr7254.BlockRequest()
        
        numData = cpSvr7254.GetHeaderValue(1)
        print(numData)
        for ixRow in range(numData):
            tempData=[]
            for ixCol in range(14):
                tempData.append(cpSvr7254.GetDataValue(ixCol, ixRow))
            data.append(tempData)
        # time.sleep(0.1)
        if (cpSvr7254.Continue == False):
            break

    return data

# 연기금 투자 상위 리스트

In [46]:
df = stock.get_market_net_purchases_of_equities_by_ticker("20211001", "20211015", "KOSPI", "연기금")
df = df.sort_values("순매수거래량", ascending = False)
print(df.head(10))

            종목명     매도거래량     매수거래량   순매수거래량         매도거래대금         매수거래대금  \
티커                                                                           
001450     현대해상    213376   1726687  1513311     5860549250    47085917800   
034020    두산중공업   2928026   4285406  1357380    62389059900    92220325550   
024110     기업은행    456558   1764478  1307920     4843634100    18706823250   
028050  삼성엔지니어링    562122   1743489  1181367    13963185550    43593879100   
395400     SK리츠      3760    868639   864879       22707560     5285660490   
003530   한화투자증권     15869    705476   689607       70636675     3180747030   
009830    한화솔루션   2080051   2738097   658046    90996990350   119813612000   
003470    유안타증권      6687    583309   576622       26347565     2357703730   
035720      카카오  12888146  13428880   540734  1502468061500  1566037708000   
005880     대한해운     66219    493961   427742      188691625     1450332980   

            순매수거래대금  
티커                   
001450  41225368550

In [47]:
df.shape

(806, 7)

In [53]:
lst = df.index
lst = ['A'+str(tic) for tic in lst]
lst = lst[:20]

In [80]:
tt = 'A001450'
tmp = []
tmp.extend([tt  for i in range(10)])
tmp = pd.Series(tmp)
tmp

0    A001450
1    A001450
2    A001450
3    A001450
4    A001450
5    A001450
6    A001450
7    A001450
8    A001450
9    A001450
dtype: object

In [77]:
tmp

In [83]:
fromDate = '20211001' # 요청 시작 날짜
toDate = '20211015'   # 요청 마지막 날짜

results = pd.DataFrame()
for code in lst:
    data = subCpSvr7254(code, fromDate, toDate)
    df=pd.DataFrame(data)
    tmp = []
    tmp.extend([code for i in range(len(df))])

    tmp  = pd.Series(tmp)
    df = pd.concat([tmp, df], axis = 1)
    df.columns = ['종목','일자', '개인', '외국인', '기관계', '금융투자', '보험', '투신', '은행', '기타금융', '연기금', '기타법인', '기타외인', '사모펀드', '국가지자체']
    results = pd.concat([results, df])

남은 횟수:  60
9
남은 횟수:  59
9
남은 횟수:  58
9
남은 횟수:  57
9
남은 횟수:  56
9
남은 횟수:  55
9
남은 횟수:  54
9
남은 횟수:  53
9
남은 횟수:  52
9
남은 횟수:  51
9
남은 횟수:  50
9
남은 횟수:  49
9
남은 횟수:  48
9
남은 횟수:  47
9
남은 횟수:  46
9
남은 횟수:  45
9
남은 횟수:  44
9
남은 횟수:  43
9
남은 횟수:  42
9
남은 횟수:  41
9


In [84]:
results.head(30)

,종목,일자,개인,외국인,기관계,금융투자,보험,투신,은행,기타금융,연기금,기타법인,기타외인,사모펀드,국가지자체
0,A001450,20211015,-2512730,615839,1927798,31449,250162,68196,-6101,12678,1513311,-32270,1363,58103,0
1,A001450,20211014,-2309930,509194,1830883,32070,229247,75523,-6794,12178,1451202,-31258,1111,37457,0
2,A001450,20211013,-2273309,472857,1838290,14833,227091,144842,-6566,10013,1414084,-37622,-216,33993,0
3,A001450,20211012,-2273521,467967,1844472,262,225091,155551,-6566,8895,1411252,-35730,-3188,49987,0
4,A001450,20211008,-2123491,532447,1610332,7779,197599,117808,-8596,5905,1240754,-26093,6805,49083,0
5,A001450,20211007,-1877976,484994,1408740,25468,166710,101520,-8596,5560,1055849,-23112,7354,62229,0
6,A001450,20211006,-1672289,509641,1177794,-1838,124899,120271,-5667,5056,880322,-22804,7658,54751,0
7,A001450,20211005,-1011275,273996,726498,-14890,83788,42111,-7332,4852,598318,1581,9200,19651,0
8,A001450,20211001,-739461,195973,546154,944,34892,28848,0,-2200,472456,-11912,9246,11214,0
0,A034020,20211015,-2512074,763912,1744127,-673138,151956,629761,1958,-900,1357380,20641,-16606,277110,0


In [81]:

if __name__ == "__main__":

    code='A005930'        # 삼성전자 코드 
    fromDate = '20211001' # 요청 시작 날짜
    toDate = '20211015'   # 요청 마지막 날짜

    # ### 자료가져오기
    data=subCpSvr7254(code, fromDate, toDate)
    # print(data)
    df=pd.DataFrame(data,  columns=['일자', '개인', '외국인', '기관계', '금융투자', '보험', '투신', '은행', '기타금융', '연기금', '기타법인', '기타외인', '사모펀드', '국가지자체'])
    # df.to_csv('subCpSvr7254.csv')

    new = []
    new.extend([code for i in range(len(df))])
    new = pd.Series(new)

    df = pd.concat([new, df], axis = 1)
    df.head()

남은 횟수:  60
9


In [82]:
df

,0,일자,개인,외국인,기관계,금융투자,보험,투신,은행,기타금융,연기금,기타법인,기타외인,사모펀드,국가지자체
0,A005930,20211015,27190554,-28655966,651298,4334798,-727412,197529,22916,-19255,-3130023,741619,72495,-27255,0
1,A005930,20211014,26367812,-27279862,195010,3708365,-666954,97884,1577,-19497,-2817623,657381,59659,-108742,0
2,A005930,20211013,24356649,-23492689,-1501652,1714730,-573478,20927,1597,-30021,-2581160,594677,43015,-54247,0
3,A005930,20211012,22132029,-18220844,-4227248,-825157,-473312,-12944,-14987,-22477,-2601215,282910,33153,-277156,0
4,A005930,20211008,9976958,-7216515,-2806046,82652,-243262,165211,-30381,38514,-2613614,34197,11406,-205166,0
5,A005930,20211007,7364103,-7536415,116985,1647696,-154713,122406,-33722,47156,-1343957,44372,10955,-167881,0
6,A005930,20211006,6097952,-4874909,-1274373,-173912,-64531,52714,-32033,41386,-936872,46879,4451,-161125,0
7,A005930,20211005,4722992,-3804470,-931906,-418029,-39596,-49836,-33702,40462,-337113,5335,8049,-94092,0
8,A005930,20211001,2939755,-778886,-2187817,-1591761,-33856,-112125,13842,49801,-441586,24224,2724,-72132,0


In [60]:
t = ['A005930']
tmp = pd.DataFrame(t*10, columns='type')
tmp

TypeError: Index(...) must be called with a collection of some kind, 'type' was passed

In [62]:
pd.Series(t*10)

0    A005930
1    A005930
2    A005930
3    A005930
4    A005930
5    A005930
6    A005930
7    A005930
8    A005930
9    A005930
dtype: object